In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from  pyspark.sql import SQLContext
from pyspark.sql.types import *

import datetime
import os
import shutil
import csv
import random
import uuid

In [ ]:
DATA_DIR = 'data'
CHECKPOINT_DIR = 'checkpoint_dir'
OUT_DIR = 'streaming_out'
def init_dir():
    shutil.rmtree(DATA_DIR, ignore_errors=True)
    shutil.rmtree(CHECKPOINT_DIR, ignore_errors = True)
    shutil.rmtree(OUT_DIR, ignore_errors = True)
    os.mkdir(DATA_DIR)
    os.mkdir(CHECKPOINT_DIR)
    os.mkdir(OUT_DIR)
init_dir()

In [ ]:
spark = SparkSession.builder \
             .master("local") \
             .appName("stream to csv")\
             .config("spark.some.config.option", "some-value") \
            .getOrCreate()


In [ ]:
dataSchema = StructType([StructField('creation_time',TimestampType(),True),
        StructField('key',LongType(),True),
        StructField('x',LongType(),True),
                        ])

In [ ]:
streaming = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1)\
.csv( DATA_DIR)
spark.conf.set("spark.sql.shuffle.partitions", 5)

In [ ]:
streaming.createOrReplaceTempView("my_table")

In [ ]:
spark.sql("select * from my_table").writeStream.trigger(processingTime='60 seconds')\
.format("parquet")\
.option("checkpointLocation", CHECKPOINT_DIR).outputMode("append")\
.option("path", OUT_DIR).start()

In [ ]:
def make_data(num_rows = 3):
    now = datetime.datetime.now()
    with open(os.path.join(DATA_DIR, f'{uuid.uuid1()}.csv'), 'w') as write_obj:
        csv_writer = csv.writer(write_obj)
        for i in range(num_rows):
            row = [now.strftime('%Y-%m-%d %H:%M:%S'), random.randint(1,10), random.randint(1, 10)]
            csv_writer.writerow(row)
        
make_data()